In [ ]:
## 패키지/라이브러리 설치
# %pip install python-docx tiktoken chromadb

In [1]:
import os
import tiktoken
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from docx import Document
from openai import OpenAI


## 1. 문서 내용 읽기 ########################################################################
document = Document('law_1.docx')

for index, paragraph in enumerate(document.paragraphs):
    print(f'{index}.{paragraph.text}')

    if index == 4:
        break
    
## 읽어 온 단락을 하나의 문서로 만들기
full_text = ''
for paragraph in document.paragraphs:
    full_text += f'{paragraph.text}'

## 2. 문서 분할(쪼개기) ######################################################################
encoding_model = tiktoken.encoding_for_model('gpt-4o')

## 토큰화
encoding = encoding_model.encode(full_text)
def split_text(full_text, chunk_size):
    encoder_model = tiktoken.encoding_for_model('gpt-4o')
    total_encoding = encoder_model.encode(full_text)
    total_token_count = len(total_encoding)

    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoding = encoding_model.decode(chunk)
        text_list.append(decoding)

    return text_list

chunk_list = split_text(full_text, 1500)

## 3. 벡터 데이터베이스에 저장 #################################################################
## 임베딩 모델 지정
OPEN_AI_KEY = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(
    api_key=OPEN_AI_KEY,
    model_name='text-embedding-3-large'
)

## 크로마 클라이언트 생성
chromadb_client = chromadb.Client()

collection_name = 'law_collection'

law_collection = chromadb_client.get_or_create_collection(
    collection_name, 
    embedding_function = openai_embedding)

## 컬렉션에 설정할 인덱스 생성
id_list = []

for index in range(len(chunk_list)):
    id_list.append(f'{index}')

law_collection.add(documents=chunk_list, ids=id_list)

## chromadb에 생성된 컬렉션 목록 확인
collections = chromadb_client.list_collections()

# 4. 질문이 있으면, 벡터 데이터베이스에서 검색  ######################################################
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 유사도 검색
retrieved_doc = law_collection.query(query_texts=query, n_results=3)

## 5. 질문 -> 답변 ####################################################################################

# [방법1]  ===============================
client = OpenAI()

response = client.responses.create(
    model="gpt-4o",
    instructions=f'''당신은 전세사기피해 법률 전문가입니다.[context]를 참고하여 사용자의 질문에 답변해주세요.
    [context]
    {retrieved_doc['documents'][0]}
    ''',

    input = query,
)

print(response.output_text)

# [방법2] ==================================
# client = OpenAI()

# response = client.responses.create(
#     model="gpt-4o",
#     input=[
#         {
#             "role": "developer",
#             "content": f'''당신은 전세사기피해 법률 전문가입니다. [context]를 참고하여 사용자의 질문에 답변해주세요.
# [context]
# {retrieved_doc['documents'][0]}''',
#         },
#         {
#             "role": "user",
#             "content": query
#         }
#     ]
# )

# print(response.output_text)

0.
1.전세사기피해자 지원 및 주거안정에 관한 특별법 ( 약칭: 전세사기피해자법 )
2.[시행 2024. 11. 11.] [법률 제20429호, 2024. 9. 10., 일부개정]
3.
4.국토교통부(피해지원총괄과) 044-201-5233, 5234
전세사기피해자에 대한 금융지원은 주로 피해자들의 주거 안정을 돕기 위해 마련된 다양한 지원책을 포함합니다. 대표적인 지원 사항은 다음과 같습니다.

1. **임대차 보증금 반환 대출**: 전세사기피해자에게 임차보증금을 반환하지 못한 경우, 필요한 경우 저리의 대출을 받을 수 있도록 지원합니다.

2. **이자 지원**: 피해자가 금융기관에서 대출받는 경우, 일정한 요건 하에 이자의 일부를 지원받을 수 있습니다.

3. **공공임대주택 우선 공급**: 주거 안정을 위해 피해자가 공공임대주택을 우선적으로 제공받도록 지원합니다.

이러한 지원 조치는 전세사기로 인한 직접적인 경제적 피해를 경감하고, 피해자의 회복을 돕기 위해 마련된 것입니다. 지원의 세부 조건 및 절차는 국토교통부와 관련 기관의 심의를 거쳐 결정됩니다.
